In [1]:
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
import pandas as pd
import astropy.coordinates
from math import *

In [2]:
# Read the Exoplanet list, skip comments
Exoplanetlist = pd.read_csv("NASA Exoplanet Archive-all.csv", skiprows=291)
Exoplanetlist = Exoplanetlist[
    ["pl_name", "gaia_id", "hostname", "glat", "glon", "ra", "dec", "sy_dist"]
]

C:\Users\ethan\AppData\Local\Temp\ipykernel_7652\3032320718.py:2: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  Exoplanetlist = pd.read_csv("NASA Exoplanet Archive-all.csv", skiprows=291)


In [3]:
Exoplanetlist.drop_duplicates(["gaia_id"], inplace=True, keep="last")
Exoplanetlist.set_index("gaia_id", drop=True, inplace=True)

In [4]:
# Set planet info
planetname = "11 Com b"
hostname = Exoplanetlist[Exoplanetlist["pl_name"] == planetname].hostname[0]
planetra = Exoplanetlist[Exoplanetlist["pl_name"] == planetname].ra[0]
planetdec = Exoplanetlist[Exoplanetlist["pl_name"] == planetname].dec[0]
planetdist = Exoplanetlist[Exoplanetlist["pl_name"] == planetname].sy_dist[0]
planetglon = Exoplanetlist[Exoplanetlist["pl_name"] == planetname].glon[0]
planetglat = Exoplanetlist[Exoplanetlist["pl_name"] == planetname].glat[0]

C:\Users\ethan\AppData\Local\Temp\ipykernel_7652\1569465981.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  hostname = Exoplanetlist[Exoplanetlist["pl_name"] == planetname].hostname[0]
C:\Users\ethan\AppData\Local\Temp\ipykernel_7652\1569465981.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  planetra = Exoplanetlist[Exoplanetlist["pl_name"] == planetname].ra[0]
C:\Users\ethan\AppData\Local\Temp\ipykernel_7652\1569465981.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To ac

In [5]:
job = Gaia.launch_job(
    f"""SELECT TOP 5000 gaia_source.designation,gaia_source.source_id,gaia_source.pseudocolour,gaia_source.l,gaia_source.b,gaia_source.grvs_mag,gaia_source.ra,gaia_source.dec,gaia_source.distance_gspphot

FROM gaiadr3.gaia_source

WHERE

CONTAINS(

	POINT('ICRS',gaiadr3.gaia_source.ra,gaiadr3.gaia_source.dec),

	CIRCLE('ICRS',{planetra},{planetdec},{u.Quantity(degrees(atan(20/planetdist)), u.deg).value})

)=1  AND  (gaiadr3.gaia_source.distance_gspphot>={planetdist-20} AND gaiadr3.gaia_source.distance_gspphot<={planetdist+20})"""
)

In [6]:
df = job.get_results().to_pandas()
df.rename(
    columns={"distance_gspphot": "distance", "l": "glon", "b": "glat"}, inplace=True
)

In [7]:
df

,designation,source_id,pseudocolour,glon,glat,grvs_mag,ra,dec,distance
0,Gaia DR3 3898405960470368768,3898405960470368768,NaN,270.035251,66.494145,10.622203,180.256084,6.897804,103.884300
1,Gaia DR3 3898424239851113600,3898424239851113600,NaN,269.203659,66.416539,13.783198,179.926325,6.993829,97.473602
2,Gaia DR3 3898658848144281984,3898658848144281984,NaN,272.690704,67.309881,9.501500,181.568500,7.128941,98.019402
3,Gaia DR3 3898663967745301760,3898663967745301760,NaN,272.647884,67.412437,11.482728,181.600410,7.227871,111.180603
4,Gaia DR3 3898735436000898688,3898735436000898688,NaN,270.496863,66.676077,10.471425,180.506481,6.968250,99.590500
...,...,...,...,...,...,...,...,...,...
1126,Gaia DR3 3737420759330432384,3737420759330432384,NaN,312.934737,75.742074,10.186790,195.376508,13.066225,76.505898
1127,Gaia DR3 3737424057865317760,3737424057865317760,NaN,313.190501,75.840524,8.923455,195.423882,13.173485,101.655296
1128,Gaia DR3 3737518959462646912,3737518959462646912,NaN,316.082122,75.197975,9.150648,196.274895,12.676143,108.693703
1129,Gaia DR3 3737840257376242816,3737840257376242816,NaN,316.618187,76.202607,9.565280,196.188909,13.685988,98.958397


In [8]:
planetcoordinates = astropy.coordinates.spherical_to_cartesian(
    planetdist, radians(planetglat), radians(planetglon)
)

In [9]:
exoplanetdata = pd.DataFrame(
    columns=["source_id", "l", "d", "pseudocolour", "brightness"]
)

for i in df.index:

    source_id = df.loc[i, "source_id"]
    pseudocolour = df.loc[i, "pseudocolour"]

    d1 = df.loc[i, "distance"]


    starcoordinates = astropy.coordinates.spherical_to_cartesian(
        d1, radians(df.loc[i, "glat"]), radians(df.loc[i, "glon"])
    )


    difference = [0, 0, 0]

    for i in range(3):

        difference[i] = starcoordinates[i] - planetcoordinates[i]


    perspectivecoordinates = astropy.coordinates.cartesian_to_spherical(
        difference[0], difference[1], difference[2]
    )


    exoplanetbrightness = (
        df.loc[i, "grvs_mag"] / (d1**2) * (perspectivecoordinates[0].value ** 2)

    )


    a = pd.DataFrame(
        {
            "source_id": source_id,
            "mag": perspectivecoordinates[0].value,
            "l": perspectivecoordinates[1].value,
            "d": perspectivecoordinates[2].value,
            "pseudocolour": [pseudocolour],
            "brightness": exoplanetbrightness,
        }
    )

    exoplanetdata = pd.concat([exoplanetdata, a])

exoplanetdata.set_index("source_id", inplace=True, drop=True)

exoplanetdata

C:\Users\ethan\AppData\Local\Temp\ipykernel_7652\167928328.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  exoplanetdata = pd.concat([exoplanetdata, a])
C:\Users\ethan\AppData\Local\Temp\ipykernel_7652\167928328.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  exoplanetdata = pd.concat([exoplanetdata, a])


,l,d,pseudocolour,brightness,mag
source_id,,,,,
3898405960470368768,0.175432,4.798822,NaN,0.467500,23.043289
3898424239851113600,-0.094195,4.781270,NaN,0.412239,20.303254
3898658848144281984,-0.041889,4.905726,NaN,0.369015,19.316936
3898663967745301760,0.441422,4.874859,NaN,0.548290,26.707800
3898735436000898688,0.010132,4.822383,NaN,0.411556,20.727002
...,...,...,...,...,...
3737420759330432384,-0.830956,0.328488,NaN,0.869437,23.142904
3737424057865317760,0.368563,0.036835,NaN,0.380061,20.331076
3737518959462646912,0.562906,6.263460,NaN,0.541297,25.943344


In [10]:
exoplanetdata = pd.DataFrame(columns=["source_id", "brightness", "pseudocolour"])
for i in df.index:
    source_id = df.loc[i, "source_id"]
    pseudocolour = df.loc[i, "pseudocolour"]
    d1 = df.loc[i, "distance"]

    starcoordinates = astropy.coordinates.spherical_to_cartesian(
        d1, radians(df.loc[i, "glat"]), radians(df.loc[i, "glon"])
    )

    difference = [0, 0, 0]
    for i in range(3):
        difference[i] = starcoordinates[i] - planetcoordinates[i]

    exoplanetbrightness = (
        df.loc[i, "grvs_mag"]
        / (d1**2)
        * (
            difference[0].value ** 2
            + difference[1].value ** 2
            + difference[2].value ** 2
        )
    )
    a = pd.DataFrame(
        {
            "source_id": source_id,
            "x": difference[0].value,
            "y": difference[1].value,
            "z": difference[2].value,
            "pseudocolour": [pseudocolour],
            "brightness": exoplanetbrightness,
        }
    )
    exoplanetdata = pd.concat([exoplanetdata, a])
exoplanetdata.set_index("source_id", inplace=True, drop=True)
exoplanetdata

C:\Users\ethan\AppData\Local\Temp\ipykernel_7652\2199179683.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  exoplanetdata = pd.concat([exoplanetdata, a])
C:\Users\ethan\AppData\Local\Temp\ipykernel_7652\2199179683.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  exoplanetdata = pd.concat([exoplanetdata, a])


,brightness,pseudocolour,x,y,z
source_id,,,,,
3898405960470368768,0.467500,NaN,1.958700,-22.604900,4.021835
3898424239851113600,0.412239,NaN,1.391207,-20.165315,-1.909638
3898658848144281984,0.369015,NaN,3.708208,-18.940403,-0.808922
3898663967745301760,0.548290,NaN,3.906037,-23.829715,11.410265
3898735436000898688,0.411556,NaN,2.275144,-20.600685,0.210002
...,...,...,...,...,...
3737420759330432384,0.869437,NaN,14.768014,5.033481,-17.092801
3737424057865317760,0.380061,NaN,18.952896,0.698446,7.324791
3737518959462646912,0.541297,NaN,21.936227,-0.432764,13.844557


In [11]:
exoplanetdata = pd.DataFrame(
    columns=["source_id", "long", "lad", "brightness", "pseudocolour"]
)

for i in df.index:

    source_id = df.loc[i, "source_id"]
    pseudocolour = df.loc[i, "pseudocolour"]

    d1 = df.loc[i, "distance"]


    diffangle = radians(abs(df.loc[i, "glat"] - planetglat))


    d2 = dist((d1 * cos(diffangle), d1 * sin(diffangle)), (planetdist, 0))

    exoplanetlongangle = pi / 2 - asin(sin(diffangle) / d2 * d1)
    exoplanetladangle = radians(df.loc[i, "glon"])


    exoplanetbrightness = df.loc[i, "grvs_mag"] / (d1**2) * (d2**2)

    a = pd.DataFrame(
        {
            "source_id": source_id,
            "long": [exoplanetlongangle],
            "lad": [exoplanetladangle],
            "brightness": [exoplanetbrightness],
            "pseudocolour": [pseudocolour],
        }
    )
    exoplanetdata = pd.concat([exoplanetdata, a])

exoplanetdata.set_index("source_id", inplace=True, drop=True)
exoplanetdata

C:\Users\ethan\AppData\Local\Temp\ipykernel_7652\2512987135.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  exoplanetdata = pd.concat([exoplanetdata, a])


,long,lad,brightness,pseudocolour
source_id,,,,
3898405960470368768,0.381377,4.713004,0.514470,NaN
3898424239851113600,0.109680,4.698490,0.589643,NaN
3898658848144281984,0.161730,4.759351,0.353274,NaN
3898663967745301760,0.651981,4.758603,0.646084,NaN
3898735436000898688,0.214784,4.721061,0.443872,NaN
...,...,...,...,...
3737420759330432384,1.371235,5.461741,0.508493,NaN
3737424057865317760,1.094000,5.466205,0.076793,NaN
3737518959462646912,1.207002,5.516674,0.209004,NaN
